In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

random.seed(42)
np.random.seed(42)

# Parameters
num_users = 700
start_date = datetime(2024, 12, 1)
end_date = datetime(2025, 2, 15)

# Generate users
users = [f"user_{i}@example.com" for i in range(1, num_users + 1)]

# Generate date range
dates = pd.date_range(start=start_date, end=end_date, freq="D")

records = []

for date in dates:
    # Random number of active users per day
    active_users = np.random.choice(
        users,
        size=np.random.randint(1, 50),
        replace=False
    )

    for user in active_users:
        # Each user can log in 1–3 times per day
        for _ in range(np.random.randint(1, 4)):
            login_time = date + timedelta(
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59),
            )
            records.append((login_time, user))

logins_df = pd.DataFrame(records, columns=["login_timestamp", "user_email"])

logins_df.head()


,login_timestamp,user_email
0,2024-12-01 20:07:01,user_681@example.com
1,2024-12-01 23:17:15,user_681@example.com
2,2024-12-01 07:08:47,user_681@example.com
3,2024-12-01 03:43:47,user_165@example.com
4,2024-12-01 17:05:37,user_55@example.com


In [11]:
logins_df.sample(10)

,login_timestamp,user_email
450,2024-12-07 15:59:45,user_447@example.com
69,2024-12-01 09:53:40,user_429@example.com
959,2024-12-18 09:31:38,user_672@example.com
2751,2025-01-22 04:27:19,user_1@example.com
1916,2025-01-05 09:08:06,user_432@example.com
2877,2025-01-28 08:33:36,user_148@example.com
3280,2025-02-03 09:38:10,user_353@example.com
1341,2024-12-25 13:06:00,user_639@example.com
653,2024-12-14 04:36:42,user_556@example.com
2287,2025-01-13 02:14:24,user_465@example.com


In [12]:
logins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4030 entries, 0 to 4029
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   login_timestamp  4030 non-null   datetime64[ns]
 1   user_email       4030 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 63.1+ KB


In [13]:
logins_df.describe()

,login_timestamp
count,4030
mean,2025-01-08 10:02:44.084119296
min,2024-12-01 00:46:56
25%,2024-12-19 21:18:54.750000128
50%,2025-01-07 19:10:59
75%,2025-01-31 06:24:26.249999872
max,2025-02-15 23:47:56


In [14]:
import duckdb

con = duckdb.connect()

# Register dataframe as table
con.execute("CREATE TABLE Logins AS SELECT * FROM logins_df")

# Quick sanity check
con.execute("""
SELECT COUNT(*) AS total_rows,
       COUNT(DISTINCT user_email) AS unique_users,
       MIN(login_timestamp),
       MAX(login_timestamp)
FROM Logins
""").fetchdf()


,total_rows,unique_users,min(login_timestamp),max(login_timestamp)
0,4030,662,2024-12-01 00:46:56,2025-02-15 23:47:56


In [18]:
result = con.execute("""
WITH date_spine AS (
    SELECT
        DATE '2025-01-01' + n * INTERVAL '1 day' AS activity_date
    FROM range(31) t(n)
),
logins_by_day AS (
    SELECT DISTINCT
        CAST(login_timestamp AS DATE) AS login_date,
        user_email
    FROM Logins
)
SELECT
    d.activity_date,
    COUNT(DISTINCT l.user_email) AS rolling_30d_unique_users
FROM date_spine d
LEFT JOIN logins_by_day l
    ON l.login_date BETWEEN d.activity_date - INTERVAL '29 days'
                         AND d.activity_date
GROUP BY d.activity_date
ORDER BY d.activity_date
""").fetchdf()

result.head(31)


,activity_date,rolling_30d_unique_users
0,2025-01-01,481
1,2025-01-02,476
2,2025-01-03,476
3,2025-01-04,472
4,2025-01-05,462
5,2025-01-06,465
6,2025-01-07,464
7,2025-01-08,468
8,2025-01-09,479
9,2025-01-10,485
